In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
from celeba_download_demo import Curdir_loader

In [3]:
list_attr_celeba_filename = 'list_attr_celeba.txt'

cl = Curdir_loader()
df = cl.load_file_asDF(list_attr_celeba_filename).replace(to_replace=-1, value=0)

In [8]:
num_data = 202599
train_size = 180000
test_size = num_data - train_size - 1

In [ ]:
class Lasso(nn.CrossEntropyLoss):

            norm  = torch.FloatTensor([0])
            for param in layers.parameters():
                norm += torch.norm(param, p=1)
            loss += lbd * norm.squeeze_()

In [75]:
def learn_and_eval(label):
    X = df.drop(labels=0, axis=0).drop(['Name'], axis=1).drop([label], axis=1)
    y = df[label]

    X_train = X.iloc[:train_size]
    y_train = y.iloc[:train_size]
    X_test = X.iloc[train_size:]
    y_test = y.iloc[train_size:]

    input_size = X_train.shape[1]
    output_size = 2

    X_train = torch.FloatTensor(X_train.values)
    y_train = nn.functional.one_hot(torch.tensor(y_train.values), num_classes=2).type(torch.FloatTensor)
    X_test = torch.FloatTensor(X_test.values)
    y_test = nn.functional.one_hot(torch.tensor(y_test.values), num_classes=2).type(torch.FloatTensor)

    layers = nn.Sequential(nn.Linear(input_size, 20), 
                        nn.ReLU(), 
                        nn.Linear(20, 10),
                        nn.ReLU(),
                        nn.Linear(10, output_size))
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(layers.parameters())

    epoch_num = 10
    batch_iter = 100
    batch_size = int(train_size / batch_iter)
    X_train_batch = X_train.split(batch_size, dim=0)
    y_train_batch = y_train.split(batch_size, dim=0)

    true_positive, true_negative, false_positive, false_negative, true = 0, 0, 0, 0, 0
    for i in range(test_size):
        if torch.argmax(y_test[i]) == 0:
            true += 1
            if torch.argmax(layers(X_test[i])) == 0:
                true_positive += 1
            else:
                false_negative += 1
        else:
            if torch.argmax(layers(X_test[i])) == 1:
                true_negative += 1
            else:
                false_positive += 1
    

    print('Primary test acc: (True  positive: {:.4f}) | (True  negative: {:.4f})'.format(true_positive / test_size, true_negative / test_size))
    print('                  (False negative: {:.4f}) | (False positive: {:.4f})'.format(false_negative / test_size, false_positive / test_size))

    print()

    lbd = 0.1
    for epoch in range(epoch_num):
        for i in range(batch_iter):
            hypo = layers(X_train_batch[i])
            loss = loss_fn(hypo, y_train_batch[i])

            loss.backward()
            optimizer.step()

        print('Epoch: {:2d} | loss: {}'.format(epoch + 1, loss.item()))
    
    true_positive, true_negative, false_positive, false_negative, true = 0, 0, 0, 0, 0
    for i in range(test_size):
        if torch.argmax(y_test[i]) == 0:
            true += 1
            if torch.argmax(layers(X_test[i])) == 0:
                true_positive += 1
            else:
                false_negative += 1
        else:
            if torch.argmax(layers(X_test[i])) == 1:
                true_negative += 1
            else:
                false_positive += 1
    

    print('\nFinal test acc: (True  positive: {:.4f}) | (True  negative: {:.4f})'.format(true_positive / test_size, true_negative / test_size))
    print('                (False negative: {:.4f}) | (False positive: {:.4f})'.format(false_negative / test_size, false_positive / test_size))

'''
    accurate = 0
    for i in range(test_size):
        if torch.argmax(layers(X_test[i])) == torch.argmax(y_test[i]):
            accurate += 1
    print('Final test acc: ', accurate / test_size)
            '''

    

    

"\n    accurate = 0\n    for i in range(test_size):\n        if torch.argmax(layers(X_test[i])) == torch.argmax(y_test[i]):\n            accurate += 1\n    print('Final test acc: ', accurate / test_size)\n            "

In [77]:
learn_and_eval('Male')

Primary test acc: (True  positive: 0.0000) | (True  negative: 0.3903)
                  (False negative: 0.6097) | (False positive: 0.0000)

Epoch:  1 | loss: 0.6916339993476868
Epoch:  2 | loss: 0.6836643218994141
Epoch:  3 | loss: 0.6841010451316833
Epoch:  4 | loss: 0.7076199650764465
Epoch:  5 | loss: 0.6865398287773132
Epoch:  6 | loss: 0.6970726251602173
Epoch:  7 | loss: 0.6956887245178223
Epoch:  8 | loss: 0.6861046552658081
Epoch:  9 | loss: 0.6964628100395203
Epoch: 10 | loss: 0.6919003129005432

Final test acc: (True  positive: 0.6030) | (True  negative: 0.0046)
                (False negative: 0.0067) | (False positive: 0.3858)


In [69]:
learn_and_eval('Heavy_Makeup')


Primary test acc: (True  positive: 0.0152) | (True  negative: 0.3911)
                  (False negative: 0.5813) | (False positive: 0.0123)

Epoch:  1 | loss: 1.1565254926681519
Epoch:  2 | loss: 1.1570155620574951
Epoch:  3 | loss: 1.2061915397644043
Epoch:  4 | loss: 1.2011985778808594
Epoch:  5 | loss: 1.1359009742736816
Epoch:  6 | loss: 1.135117769241333
Epoch:  7 | loss: 1.099778413772583
Epoch:  8 | loss: 1.1018284559249878
Epoch:  9 | loss: 1.1267027854919434
Epoch: 10 | loss: 1.1535048484802246

Final test acc: (True  positive: 0.5966) | (True  negative: 0.0000)
                (False negative: 0.0000) | (False positive: 0.4034)


In [78]:
learn_and_eval('Young')

Primary test acc: (True  positive: 0.2430) | (True  negative: 0.0000)
                  (False negative: 0.0000) | (False positive: 0.7570)

Epoch:  1 | loss: 0.6602745652198792
Epoch:  2 | loss: 0.5876858830451965
Epoch:  3 | loss: 0.5836073160171509
Epoch:  4 | loss: 0.570396363735199
Epoch:  5 | loss: 0.6058144569396973
Epoch:  6 | loss: 0.5647022724151611
Epoch:  7 | loss: 0.568134069442749
Epoch:  8 | loss: 0.5794580578804016
Epoch:  9 | loss: 0.5726785659790039
Epoch: 10 | loss: 0.5667760968208313

Final test acc: (True  positive: 0.0000) | (True  negative: 0.7570)
                (False negative: 0.2430) | (False positive: 0.0000)


In [79]:
learn_and_eval('Bald')

Primary test acc: (True  positive: 0.0000) | (True  negative: 0.0208)
                  (False negative: 0.9792) | (False positive: 0.0000)

Epoch:  1 | loss: 0.11429860442876816
Epoch:  2 | loss: 0.269246369600296
Epoch:  3 | loss: 0.1173100695014
Epoch:  4 | loss: 0.10979069769382477
Epoch:  5 | loss: 0.15397405624389648
Epoch:  6 | loss: 0.14083194732666016
Epoch:  7 | loss: 0.09608408808708191
Epoch:  8 | loss: 0.11256446689367294
Epoch:  9 | loss: 0.11404537409543991
Epoch: 10 | loss: 0.09765629470348358

Final test acc: (True  positive: 0.9792) | (True  negative: 0.0000)
                (False negative: 0.0000) | (False positive: 0.0208)


In [81]:
learn_and_eval('5_o_Clock_Shadow')

Primary test acc: (True  positive: 0.5056) | (True  negative: 0.0439)
                  (False negative: 0.3923) | (False positive: 0.0582)

Epoch:  1 | loss: 0.6394578814506531
Epoch:  2 | loss: 0.4878097176551819
Epoch:  3 | loss: 0.40749216079711914
Epoch:  4 | loss: 0.4185389280319214
Epoch:  5 | loss: 0.40696221590042114
Epoch:  6 | loss: 0.39610716700553894
Epoch:  7 | loss: 0.38670966029167175
Epoch:  8 | loss: 0.3885786235332489
Epoch:  9 | loss: 0.3973217308521271
Epoch: 10 | loss: 0.40986335277557373

Final test acc: (True  positive: 0.8979) | (True  negative: 0.0000)
                (False negative: 0.0000) | (False positive: 0.1021)
